In [167]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [168]:
# Set the random seed for reproducibility
def set_random_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_random_seed(13)

In [169]:
# Set the path to the folder containing the image datasets
data_path = 'C:/Users/ashiv/Downloads/3 icon predication and count/train_data'

In [170]:
# Define the image dimensions and other parameters
image_width, image_height = 224, 224
batch_size = 32
num_classes = 7

In [89]:
# Use the ImageDataGenerator to preprocess the images and generate batches of data
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1  # Change the size of the validation dataset here.
)

train_generator = data_generator.flow_from_directory(
    data_path,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    data_path,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 174 images belonging to 7 classes.
Found 16 images belonging to 7 classes.


In [90]:
# Getting the pretrained mobile net model to finetune it for our dataset.
base_model = MobileNet(include_top=False, input_shape=(image_width, image_height, 3))

In [91]:
# Modifying the output nodes to fit for only 7 classes that we have.
# i.e [bar, cherry, rhino, clown, 7, grape, bell]
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [92]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [95]:
# Train the model
epochs = 7
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/7
6/6 [==============================] - 11s 2s/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 4.1268 - val_accuracy: 0.5000
Epoch 2/7
6/6 [==============================] - 12s 2s/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 4.5001 - val_accuracy: 0.5625
Epoch 3/7
6/6 [==============================] - 12s 2s/step - loss: 4.8497e-04 - accuracy: 1.0000 - val_loss: 4.1276 - val_accuracy: 0.5625
Epoch 4/7
6/6 [==============================] - 14s 2s/step - loss: 0.0664 - accuracy: 0.9885 - val_loss: 1.0156 - val_accuracy: 0.8750
Epoch 5/7
6/6 [==============================] - 13s 2s/step - loss: 0.2703 - accuracy: 0.9598 - val_loss: 10.8508 - val_accuracy: 0.4375
Epoch 6/7
6/6 [==============================] - 12s 2s/step - loss: 0.0355 - accuracy: 0.9770 - val_loss: 8.5931 - val_accuracy: 0.4375
Epoch 7/7
6/6 [==============================] - 12s 2s/step - loss: 0.0251 - accuracy: 0.9943 - val_loss: 2.1469 - val_accuracy: 0.8125


In [96]:
# Save the trained model
model.save('model.h5')

In [97]:
# Step 7: Loading the Trained Model
loaded_model = tf.keras.models.load_model("model.h5")
print("Trained model loaded successfully.")

Trained model loaded successfully.


In [158]:
# Set the path to the test image
image_path = 'C:/Users/ashiv/Downloads/3 icon predication and count/train_data/clown/P-1.png'

In [159]:
# Load and preprocess the test image
test_image = image.load_img(image_path, target_size=(image_width, image_height))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0  # Normalize the image

In [160]:
# Make predictions
predictions = loaded_model.predict(test_image)
class_names = train_generator.class_indices
predicted_class = np.argmax(predictions)

1/1 [==============================] - 0s 49ms/step


In [161]:
# Map the predicted class index to class names
for key, value in class_names.items():
    if value == predicted_class:
        predicted_class_name = key
        break

In [162]:
# Print the predicted class
print("Predicted class:", predicted_class_name)

Predicted class: clown


In [163]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\ashiv\AppData\Local\Temp\tmpbexglvf3\assets


INFO:tensorflow:Assets written to: C:\Users\ashiv\AppData\Local\Temp\tmpbexglvf3\assets


In [164]:
# Save the TFLite model
tflite_model_path = 'model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)